# Groundwater Flow Velocity
<p>Determining the velocity of groundwater flow is important for describing solute transport, as advection occurs with the bulk motion groundwater flow. Flow is described using the <b>Darcy velocity</b> and <b>porosity</b>:</p>
<h3><center>$ \bar{u} = \frac{q}{\theta}$</center></h3>
<p>This equation can be used to solve for the time that a contaminant spill would take to arrive at a well. In order to do so, we must know the path that groundwater would take, and therefore the distance of the path. If we knew, the distance, the time between a spill and chemicals showing up at the well would be:</p>
<h3><center>$t = \frac{d \theta}{q} $</center></h3>
<p>Where $d$ is distance, $\theta$ is porosity, and $q$ is Darcy velocity.</p>

In [1]:
def averageVelocity(q, theta):
    '''
    Returns the average velocity, for both single values and an array
    '''
    return q/theta

def soluteTransportTime(q, theta, d):
    '''
    Returns the average solute transport time. 
    It does not take into account diffusion or dispersion
    '''
    return (d*theta)/q

## Determining flow path
<p>To find the flow path, I start at a single position. At this point, I take a step towards the direction of lowest descent. I keep doing this until I reach a point where all points around me are either less than or equal to the current value. Additionally, at each point along the path, I record the position of each subsequent step.</p>

In [ ]:
def flowPath

## More about diffusion and dispersion
### Diffusion
<p><b>Fick's Law</b> is the equation that describes the diffusion of solute over distance:</p>
<h3><center>$ J = -D \frac{dC}{dx} $</center></h3>
<p>Where $J$ is <b>diffusion flux</b>, which is the amount of substance passing through a unit area per unit time, $D$ is diffusivity, with units $\frac{A}{T}$, and $C$ is concentration. In a more general sense, the equation can be written as:</p>
<h3><center>$J = -D \nabla C$</center></h3>

## Solute Transport
<p>Solute transport in the ground occurs by three methods: advection, diffusion, and dispersion.</p>
-<b>Advection</b> is the process of carrying the solute along with bulk fluid motion.<br>
-<b>Diffusion</b> is the process of regions of higher solute concentration flowing to lower ones. It is a <u>microscopic</u> process that is the result of random molecular motion.<br>
-<b>Dispersion</b> is similar to diffusion, but it is a <u>macroscopic</u> process. It is the result of <u>non-ideal</u> flow patterns, such as the flow between the grains within soil.

<p>These three concepts are incorporated into the <b>Advection-dispersion Reaction (ADR) equation:</b></p>
<h3><center>$\frac{\partial C}{\partial t} = D_{x} \frac{\partial^{2} C}{\partial x^{2}} + D_{y} \frac{\partial^{2} C}{\partial y^{2}} + D_{z} \frac{\partial^{2} C}{\partial z^{2}} - \nu \frac{\partial C}{\partial x} - \lambda RC$</center></h3>
<p>Where $D_{x}$, $D_{y}$, and $D_{z}$ are the hydrodynamic dispersive coefficients in each direction (how quickly solute disperses), $\nu$ is the advective velocity  in the <b>x</b> direction, $lambda$ is a decay rate, which encompasses biotic and abiotic processes that decrease solute amount, and $R$ is the <u>linear</u> equilibrium retardation factor, which is something else that I don't really know.</p>

### Dispersion coefficients
<p>Each of the dispersion coefficients is covers both diffusion and dispersion, and can therefore be broken up into its components:</p>
<h3><center>$D = D_{m} + \nu \alpha$</center></h3>
<p>Where $D_{m}$ is the <b>molecular diffusion coefficient</b> and $\alpha$ is the dispersivity, and is defined in the longitudinal, transverse, and vertical directions.</p>
<p>In order to describe these things, though, we need information on the velocity of the flow at each point to describe stuff more</p>